In [1]:
%pip install kagglehub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 638.7 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.8/178.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.6/432.6 kB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.8/8

## Downloading Dataset

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saurabhbagchi/books-dataset")

print("Path to dataset files:", path)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/ankanmahapatra/.cache/kagglehub/datasets/saurabhbagchi/books-dataset/versions/1


In [2]:
## Show the dataset files

import os

path += "/books_data"
os.listdir(path)

['users.csv', 'ratings.csv', 'books.csv']

## Checking the csv file

In [3]:
with open(path + "/books.csv", errors="ignore") as f:
    print(f.read()[:1000])

"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"
"0195153448";"Classical Mythology";"Mark P. O. Morford";"2002";"Oxford University Press";"http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg"
"0002005018";"Clara Callan";"Richard Bruce Wright";"2001";"HarperFlamingo Canada";"http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg"
"0060973129";"Decision in Normandy";"Carlo D'Este";"1991";"HarperPerennial";"http://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0060973129.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0060973129.01.LZZZZZZZ.jpg"
"0374157065";"Flu: The Story of the Great Influenza Pandemic of 1918 and the

## Loading CSV into Dataframe, finally saving it into SQLite DB

In [24]:
%pip install chardet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 1.1 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 24.1.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import chardet

with open(path + "/books.csv", "rb") as f:
    raw_data = f.read()
    detected = chardet.detect(raw_data)
    encoding = detected["encoding"]

In [6]:
encoding

'ISO-8859-1'

In [7]:
import pandas as pd
import sqlite3

# Load CSV file into a DataFrame
df = pd.read_csv(path + "/books.csv", encoding=encoding, delimiter=";", on_bad_lines="skip")

# Connect to SQLite database (or create one)
conn = sqlite3.connect("books.db")

# Write the DataFrame to an SQLite table
df.to_sql("books", conn, if_exists="replace", index=False)

# Close the connection
conn.close()

print("CSV file successfully converted to SQLite table!")


/var/folders/w2/qfwv8k7j63b5gvsyfr80f6xw0000gn/T/ipykernel_18208/1000044441.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + "/books.csv", encoding=encoding, delimiter=";", on_bad_lines="skip")


CSV file successfully converted to SQLite table!


In [8]:
df.size

2170880

## Checking if table is added into DB

In [9]:
import sqlite3

conn = sqlite3.connect("books.db")
cursor = conn.cursor()
cursor.execute("SELECT `Book-Title` FROM books LIMIT 100;")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

('Classical Mythology',)
('Clara Callan',)
('Decision in Normandy',)
('Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It',)
('The Mummies of Urumchi',)
("The Kitchen God's Wife",)
("What If?: The World's Foremost Military Historians Imagine What Might Have Been",)
('PLEADING GUILTY',)
('Under the Black Flag: The Romance and the Reality of Life Among the Pirates',)
("Where You'll Find Me: And Other Stories",)
('Nights Below Station Street',)
("Hitler's Secret Bankers: The Myth of Swiss Neutrality During the Holocaust",)
('The Middle Stories',)
('Jane Doe',)
("A Second Chicken Soup for the Woman's Soul (Chicken Soup for the Soul Series)",)
('The Witchfinder (Amos Walker Mystery Series)',)
('More Cunning Than Man: A Social History of Rats and Man',)
('Goodbye to the Buttermilk Sky',)
('The Testament',)
('Beloved (Plume Contemporary Fiction)',)
("Our Dumb Century: The Onion Presents 100 Years of Headlines from America's Finest News Source",)

## Testing basic queries

In [10]:
## Search for Title
import sqlite3

conn = sqlite3.connect("books.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM books WHERE `Book-Title` LIKE '%splendid s%' LIMIT 10;")

rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

('0060175702', 'Empress of the Splendid Season', 'Oscar Hijuelos', '1999', 'HarperCollins Publishers', 'http://images.amazon.com/images/P/0060175702.01.THUMBZZZ.jpg', 'http://images.amazon.com/images/P/0060175702.01.MZZZZZZZ.jpg', 'http://images.amazon.com/images/P/0060175702.01.LZZZZZZZ.jpg')
('0061014184', 'The Empress of the Splendid Season', 'Oscar Hijuelos', '1999', 'HarperCollins Publishers', 'http://images.amazon.com/images/P/0061014184.01.THUMBZZZ.jpg', 'http://images.amazon.com/images/P/0061014184.01.MZZZZZZZ.jpg', 'http://images.amazon.com/images/P/0061014184.01.LZZZZZZZ.jpg')
('1582700494', 'Celebrating Time Alone: Stories of Splendid Solitude', 'Lionel Fisher', '2001', 'Beyond Words Publishing', 'http://images.amazon.com/images/P/1582700494.01.THUMBZZZ.jpg', 'http://images.amazon.com/images/P/1582700494.01.MZZZZZZZ.jpg', 'http://images.amazon.com/images/P/1582700494.01.LZZZZZZZ.jpg')
('0060928700', 'Empress of the Splendid Season', 'Oscar Hijuelos', '2000', 'Perennial', 'ht

In [11]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///books.db")

db.get_table_info()

'\nCREATE TABLE books (\n\t"ISBN" TEXT, \n\t"Book-Title" TEXT, \n\t"Book-Author" TEXT, \n\t"Year-Of-Publication" TEXT, \n\t"Publisher" TEXT, \n\t"Image-URL-S" TEXT, \n\t"Image-URL-M" TEXT, \n\t"Image-URL-L" TEXT\n)\n\n/*\n3 rows from books table:\nISBN\tBook-Title\tBook-Author\tYear-Of-Publication\tPublisher\tImage-URL-S\tImage-URL-M\tImage-URL-L\n0195153448\tClassical Mythology\tMark P. O. Morford\t2002\tOxford University Press\thttp://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg\thttp://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg\thttp://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg\n0002005018\tClara Callan\tRichard Bruce Wright\t2001\tHarperFlamingo Canada\thttp://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg\thttp://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg\thttp://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg\n0060973129\tDecision in Normandy\tCarlo D\'Este\t1991\tHarperPerennial\thttp://images.amazon.com/images/P/0060973129.